### Obteniendo coordenadas de direcciones físicas con la API de google Geocoding

Es posible aprovechar la API de google Geocoding para obtener las coordenada (latitud y longitud) de una dirección. Por ejemplo queremos obtener las coordenada de CARRERA 81C CALLE 4C-61, CARTAGENA, BOLIVAR para poder colocarlas en un mapa para visualización. Es muy frecuente que tengamos miles de direcciones para encontrar sus coordenadas y hacerlo manuelamente es un atarea tediosa. Por lo tanto utilizar la API de google puede ser de gran ayuda.

Librerias a utiliazar

In [54]:
import pandas as pd
import googlemaps
import re

In [57]:
## funciones de limpieza de las direcciones

def recodificarCalle(direccion):
    expresion="[C][L][\s*]|[C][L][L][E][\s*]|[C][L][L][\s*]|[C][l][l][\s*]"
    recod=re.sub(expresion,"CALLE ",direccion)
    return recod

def recodificarCarrera(direccion):
    expresion="[C][R][\s*]|[K][R][A][\s*]|[K][A][\s*]|[C][A][\s*]|[C][R][A][\s*]|[C][r][a][\s*]"
    recod=re.sub(expresion,"CARRERA ",direccion)
    return recod

def recodificarLocal(direccion):
    expresion="[L][C][\s*]|[l][c][\s*]|[L][C][.][\s*]"
    recod=re.sub(expresion,"LOCAL ",direccion)
    return recod
    
def recodificarAvenida(direccion):
    expresion="[A][V][\s*]|[a][v][\s*]"
    recod=re.sub(expresion,"AVENIDA ",direccion)
    return recod
	
def recodificarDiagonal(direccion):
    expresion="[D][G][\s*]|[d][g][\s*]|[D][G][.][\s*]"
    recod=re.sub(expresion,"DIAGONAL ",direccion)
    return recod
	
def recodificarManzana(direccion):
    expresion="[M][Z][\s*]|[m][z][\s*]"
    recod=re.sub(expresion,"MANZANA ",direccion)
    return recod
	
def recodificarVereda(direccion):
    expresion="[V][R][D][\s*]|[m][z][\s*]"
    recod=re.sub(expresion,"VEREDA ",direccion)
    return recod

def limpiarTexto(texto):
    texto_limpio=texto.lstrip().rstrip()
    return texto_limpio

### funcion para obtener la latitud y la longitud, el try except es necesario para no interrumpir la ejecución en caso de no encontrar coordenadas para la direccion.
def obtenerCoordenadas(add):
    g = key_google.geocode(add)
    try:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    except IndexError:
        lat='null'
        lng='null'

# Funcion consolidada para limpieza previa de la direccion.
def procesarDireccion(direccion):
    dir_clean=limpiarTexto(direccion)
    dir_mod=recodificarVereda(recodificarManzana(recodificarDiagonal(
        recodificarAvenida(recodificarLocal(recodificarCarrera(
            recodificarCalle(dir_clean)))))))
    return dir_mod


La primer paso para consumir la API de google es abrir una cuenta en Google Cloud, crear un proyeto y en este habilitar la API geocoding.
https://developers.google.com/maps/documentation/geocoding/get-api-key

Una vez contamos con la API podemos hacer el llamado de la misma para poder utilziar la API:

In [44]:
key_google = googlemaps.Client(key="********************************")

## lectura de los corresponsales propios y en alianza

In [63]:
df_corresponsales=pd.read_excel('direcciones fisicas.xlsx', sheet_name='Corresponsales')

In [66]:
df_corresponsales.columns=['DEPARTAMENTO','MUNICIPIO','ESTABLECIMIENTO','DIRECCION','BARRIO','TRANSACCIONES_HABILITADAS','ALIANZA']

In [75]:
df_corresponsales2=df_corresponsales[['DEPARTAMENTO','MUNICIPIO','DIRECCION']]
df_corresponsales2

,DEPARTAMENTO,MUNICIPIO,DIRECCION
0,LA GUAJIRA,MAICAO,CL 15 15 15 BRR CENTRO
1,LA GUAJIRA,MAICAO,CL 15 15 15 LC 1
2,LA GUAJIRA,MAICAO,CL 15 15 15 LC 2
3,LA GUAJIRA,MAICAO,CL 15 13 40 L 3 EDF CAMANI
4,LA GUAJIRA,MAICAO,CR 12 14 60 LC 8 EDIF WAKED
...,...,...,...
23948,VALLE DEL CAUCA,CANDELARIA,CR 10 #14-28 AP 2
23949,CASANARE,VILLANUEVA,CL 7 #11-47
23950,META,VILLAVICENCIO,CL 53 Sur # 42-20
23951,META,VISTA HERMOSA,CL 10 # 11 - 26


## PRUEBAS

In [76]:
texto="AVENIDA SAN MARTIN CARRERA 2 # 8-142 LOCAL 101, CARTAGENA, BOLIVAR"
procesarDireccion(texto)

'AVENIDA SAN MARTIN CARRERA 2 # 8-142 LOCAL 101, CARTAGENA, BOLIVAR'

In [77]:
lista_dir=[texto]
df=pd.DataFrame({'direcciones':lista_dir})
df

,direcciones
0,AVENIDA SAN MARTIN CARRERA 2 # 8-142 LOCAL 101...


In [78]:
df['direccion_cod'] = df['direcciones'].apply(obtenerCoordenadas)
df

,direcciones,direccion_cod
0,AVENIDA SAN MARTIN CARRERA 2 # 8-142 LOCAL 101...,"(10.403636, -75.55433699999999)"


## creacion de la columna direccion

In [80]:
df_corresponsales2['dirFormateada']=df_corresponsales['DIRECCION'].apply(procesarDireccion)

<ipython-input-80-e8b9d942bfd1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corresponsales2['dirFormateada']=df_corresponsales['DIRECCION'].apply(procesarDireccion)


In [81]:
df_corresponsales2

,DEPARTAMENTO,MUNICIPIO,DIRECCION,dirFormateada
0,LA GUAJIRA,MAICAO,CL 15 15 15 BRR CENTRO,CALLE 15 15 15 BRR CENTRO
1,LA GUAJIRA,MAICAO,CL 15 15 15 LC 1,CALLE 15 15 15 LOCAL 1
2,LA GUAJIRA,MAICAO,CL 15 15 15 LC 2,CALLE 15 15 15 LOCAL 2
3,LA GUAJIRA,MAICAO,CL 15 13 40 L 3 EDF CAMANI,CALLE 15 13 40 L 3 EDF CAMANI
4,LA GUAJIRA,MAICAO,CR 12 14 60 LC 8 EDIF WAKED,CARRERA 12 14 60 LOCAL 8 EDIF WAKED
...,...,...,...,...
23948,VALLE DEL CAUCA,CANDELARIA,CR 10 #14-28 AP 2,CARRERA 10 #14-28 AP 2
23949,CASANARE,VILLANUEVA,CL 7 #11-47,CALLE 7 #11-47
23950,META,VILLAVICENCIO,CL 53 Sur # 42-20,CALLE 53 Sur # 42-20
23951,META,VISTA HERMOSA,CL 10 # 11 - 26,CALLE 10 # 11 - 26


In [82]:
df_corresponsales2['DireccionConcatenada']=df_corresponsales2['dirFormateada']+", "+df_corresponsales2['MUNICIPIO']+", "+df_corresponsales2['DEPARTAMENTO']

<ipython-input-82-cb92b6ff3a9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corresponsales2['DireccionConcatenada']=df_corresponsales2['dirFormateada']+", "+df_corresponsales2['MUNICIPIO']+", "+df_corresponsales2['DEPARTAMENTO']


In [83]:
df_corresponsales2

,DEPARTAMENTO,MUNICIPIO,DIRECCION,dirFormateada,DireccionConcatenada
0,LA GUAJIRA,MAICAO,CL 15 15 15 BRR CENTRO,CALLE 15 15 15 BRR CENTRO,"CALLE 15 15 15 BRR CENTRO, MAICAO, LA GUAJIRA"
1,LA GUAJIRA,MAICAO,CL 15 15 15 LC 1,CALLE 15 15 15 LOCAL 1,"CALLE 15 15 15 LOCAL 1, MAICAO, LA GUAJIRA"
2,LA GUAJIRA,MAICAO,CL 15 15 15 LC 2,CALLE 15 15 15 LOCAL 2,"CALLE 15 15 15 LOCAL 2, MAICAO, LA GUAJIRA"
3,LA GUAJIRA,MAICAO,CL 15 13 40 L 3 EDF CAMANI,CALLE 15 13 40 L 3 EDF CAMANI,"CALLE 15 13 40 L 3 EDF CAMANI, MAICAO, LA GUAJIRA"
4,LA GUAJIRA,MAICAO,CR 12 14 60 LC 8 EDIF WAKED,CARRERA 12 14 60 LOCAL 8 EDIF WAKED,"CARRERA 12 14 60 LOCAL 8 EDIF WAKED, MAICAO, L..."
...,...,...,...,...,...
23948,VALLE DEL CAUCA,CANDELARIA,CR 10 #14-28 AP 2,CARRERA 10 #14-28 AP 2,"CARRERA 10 #14-28 AP 2, CANDELARIA, VALLE DEL ..."
23949,CASANARE,VILLANUEVA,CL 7 #11-47,CALLE 7 #11-47,"CALLE 7 #11-47, VILLANUEVA, CASANARE"
23950,META,VILLAVICENCIO,CL 53 Sur # 42-20,CALLE 53 Sur # 42-20,"CALLE 53 Sur # 42-20, VILLAVICENCIO, META"
23951,META,VISTA HERMOSA,CL 10 # 11 - 26,CALLE 10 # 11 - 26,"CALLE 10 # 11 - 26, VISTA HERMOSA, META"


## Encontrar las coordenadas para todo el dataframe

para encontrar las coordenadas para cada una de las direcciones en el dataframe aplicamos la función a la columna correspondiente con APPLY

In [121]:
df_corresponsales2['coordenadas'] = df_corresponsales2['DireccionConcatenada'].apply(obtenerCoordenadas)

<ipython-input-121-d56927a7095c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bloque1['coordenadas'] = bloque1['DireccionConcatenada'].apply(geocode)


Una vez que se han encontrado todas las direcciones fisicas, podemos exportar el archivo en formato xlsx

In [136]:
df_corresponsales2.to_excel("CoordenadasaCorresponsales20221023.xlsx")